In [1]:

%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab
import matplotlib.pyplot as plt
import scipy.stats as stats
import datetime
import itertools

import sklearn
from sklearn.preprocessing import PowerTransformer
from scipy.stats import boxcox 

import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller, kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf

import warnings
from math import sqrt
from pandas import read_csv
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import pmdarima as pm



from pandas.tseries.offsets import DateOffset
import warnings

In [2]:
df = pd.read_excel('dataset.xlsx', index_col='Region/Country/Economy')
df.replace('..', None, inplace= True)
df.drop(columns=2022, inplace= True)
df.fillna(method='bfill', axis=1, inplace=True)
df.fillna(method='ffill', axis=1, inplace=True)
df.rename_axis('Country', inplace=True)
df = df.transpose()
df.head(2)

Country,Canada,Chile,Colombia,Costa Rica,Mexico,United States,OECD Americas,Australia,Israel,Japan,...,IEA/Accession/Association,European Union - 27,G7,G8,G20,Africa,Americas,Asia,Europe,Oceania
1971,57.486,57.724,45.999,37.704,52.102,64.525,63.486,66.342,57.262,67.048,...,62.130,57.731,66.227,58.222,57.0,30.899,61.609,54.735,58.909,64.109
1972,56.054,55.782,47.074,39.491,53.482,64.879,63.696,66.669,62.338,65.907,...,61.937,57.731,65.845,58.222,57.0,31.526,61.838,54.735,58.909,64.412


In [3]:
df1 = df.copy()

In [4]:
df['Costa Rica']

1971    37.704
1972    39.491
1973    41.514
1974    41.073
1975    41.788
1976    42.951
1977    46.696
1978    46.237
1979    44.518
1980    41.071
1981    40.896
1982    35.018
1983    36.549
1984    35.939
1985    36.863
1986    37.114
1987    37.310
1988    38.158
1989    39.003
1990    37.081
1991    37.040
1992    39.983
1993    38.817
1994    45.970
1995    44.968
1996    42.077
1997    40.760
1998    45.604
1999    41.473
2000    37.396
2001    40.601
2002    39.252
2003    39.016
2004    32.834
2005    33.626
2006    33.967
2007    35.894
2008    35.717
2009    33.730
2010    34.665
2011    35.923
2012    34.736
2013    34.451
2014    34.139
2015    33.475
2016    36.998
2017    38.268
2018    38.804
2019    35.953
2020    32.985
2021    35.295
Name: Costa Rica, dtype: float64

In [5]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(train, test, arima_order):
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    # calculate out of sample error
    rmse = sqrt(mean_squared_error(test, predictions))
    return rmse

In [6]:
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(train, test, p_values, d_values, q_values):
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p, d, q)
                try:
                    rmse = evaluate_arima_model(train, test, order)
                    if rmse < best_score:
                        best_score, best_cfg = rmse, order
                except Exception as e:
                    print(f"Exception for order {order}: {e}")
                    continue
    return best_cfg



final_model

In [8]:
df.columns

Index(['Canada', 'Chile', 'Colombia', 'Costa Rica', 'Mexico', 'United States',
       'OECD Americas', 'Australia', 'Israel', 'Japan',
       ...
       'IEA/Accession/Association', 'European Union - 27', 'G7', 'G8', 'G20',
       'Africa', 'Americas', 'Asia', 'Europe', 'Oceania'],
      dtype='object', name='Country', length=174)

In [9]:
countries = [country for country in df.columns]

In [25]:
def forecasting_CO2(country):
    if country in countries:
        pass
    else:
        print('Input a valid Country.')
    data = df[country]
    data = pd.DataFrame(data)
    data.index = pd.to_datetime(data.index, format = '%Y')
    train_data = data[:40]
    test_data = data[40:]


    warnings.filterwarnings("ignore")
    best_cfg = evaluate_models(train_data.values, test_data.values, range(0, 5), range(0, 5), range(0, 5))
    model_arima = ARIMA(data[country],order = best_cfg)
    model_arima = model_arima.fit()
    

    start = len(train_data)
    end = len(train_data)+len(test_data)-1
    pred_arima = model_arima.predict(start=start,end=end)
    

    years = 11
    future_dates=[data.index[-1]+ DateOffset(years=x)for x in range(0,years)]
    predicted_df =pd.DataFrame(index=future_dates[1:],columns=data.columns)
    

    start = len(data)
    end = (start + years)
    predicted_df[country] = model_arima.predict(start = start, end = end, dynamic= True)

    return predicted_df

In [22]:
forecasting_CO2('Canada')

,Canada
2022-01-01,41.268225
2023-01-01,40.735732
2024-01-01,40.430071
2025-01-01,40.080198
2026-01-01,39.774660
2027-01-01,39.504664
2028-01-01,39.251843
2029-01-01,39.026191
2030-01-01,38.820377
2031-01-01,38.633426


In [24]:
result_dfs = []
# Loop over countries and run the function
for country in countries:
    try:
        result_df = forecasting_CO2(country)
        if result_df is not None:
            result_dfs.append(result_df)
    except Exception as e:
        print(f"An error occurred for country {country}: {e}")

# Concatenate all result DataFrames into a single DataFrame
final_result = pd.concat(result_dfs, axis=1)

# Print or manipulate the final_result DataFrame as needed
print(final_result)








An error occurred for country Belgium: LU decomposition error.
An error occurred for country Africa: SARIMAX models require univariate `endog`. Got shape (51, 2).
An error occurred for country Africa: SARIMAX models require univariate `endog`. Got shape (51, 2).
               Canada      Chile   Colombia  Costa Rica     Mexico  \
2022-01-01  41.268225  50.303228  44.049523   35.316697  50.181888   
2023-01-01  40.735732  51.043024  43.554366   35.295199  50.220843   
2024-01-01  40.430071  50.469171  43.683264   35.316559  51.496959   
2025-01-01  40.080198  50.936281  43.604053   35.295399  52.157101   
2026-01-01  39.774660  50.539440  43.594976   35.316420  52.361625   
2027-01-01  39.504664  50.888648  43.611884   35.295598  52.685219   
2028-01-01  39.251843  50.573053  43.585964   35.316282  52.986351   
2029-01-01  39.026191  50.863875  43.604706   35.295798  53.155029   
2030-01-01  38.820377  50.592286  43.591031   35.316143  53.283104   
2031-01-01  38.633426  50.848263  43.

In [41]:
final_result = pd.concat(result_dfs, axis = 1)

In [42]:
final_result.to_csv('Forecasted_CO2.csv')